In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd ../x64
%ls 

/content/drive/My Drive/CSC413/x64
Imagenet64_train_part1.zip  Imagenet64_train_part2.zip  Imagenet64_val.zip


In [2]:
import zipfile
import os

In [ ]:
from os import walk
_, _, filenames = next(walk("."))

for filename in filenames: 
  print(filename)
  with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(".")

In [36]:
import torch 
import torchvision
import pickle
import numpy as np 
import torch.optim

In [ ]:
class generator (torch.nn.Module):

  def __init__( self):
    super(generator, self).__init__()
    self.Conv1 = torch.nn.Conv2d(3, 64, 9)
    self.BConv1 = torch.nn.Conv2d(64, 64, 3)
    self.BConv2 = torch.nn.Conv2d(64, 64, 3)
    self.BbatchNorm = torch.nn.BatchNorm2d(64)
    self.Conv2 = torch.nn.Conv2d(64, 64, 3)
    self.Conv3 = torch.nn.Conv2d(64, 256, 3)
    self.Conv4 = torch.nn.Conv2d(64, 256, 3)
    self.ConvFinal = torch.nn.Conv2d(256, 3, 9)
    self.PixelShuffle = torch.nn.PixelShuffle(upscale_factor=2)
    self.PreLU = torch.nn.PReLU();
    self.BatchNorm = torch.nn.BatchNorm2d(num_features=64)


  def forward(self, x ):
    x = self.Conv1(x)
    x = self.PreLU(x)
    x1 = x.detach().clone()
    x2 = x.detach().clone()

    for i in range(4):
      x  = self.BConv1(x)
      x = self.BatchNorm(x)
      x = self.PreLU(x)
      x = self.BConv2(x)
      x = self.BatchNorm(x)
      x = x + x1 
      x1 = x.detach().clone()

    x = self.Conv2(x)
    x = self.BatchNorm(x)
    x = x + x2 

    x = self.Conv3(x)
    x = self.PixelShuffle(x)
    x = self.PreLU(x)
    
    x = self.Conv4(x)
    x = self.PixelShuffle(x)
    x = self.PreLU(x)

    x = self.ConvFinal(x)

In [ ]:
class discrimator (torch.nn.Module):

  def __init__(self):
    super(discrimator, self).__init__()
    self.Conv1 = torch.nn.Conv2d(3, 64, 3)
    self.Conv2 = torch.nn.Conv2d(64, 64, 3, stride=2)
    self.BatchNorm2 = torch.nn.BatchNorm2d(num_features=64)
    self.Conv3 = torch.nn.Conv2d(64, 128, 3)
    self.BatchNorm3 = torch.nn.BatchNorm2d(num_features=128)
    self.Conv4 = torch.nn.Conv2d(1283, 128, 3, 2)
    self.BatchNorm4 = torch.nn.BatchNorm2d(num_features=128)
    self.Conv5 = torch.nn.Conv2d(128, 256, kernel_size=3)
    self.BatchNorm5 = torch.nn.BatchNorm2d(num_features=256)
    self.Conv6 = torch.nn.Conv2d(256, 256, kernel_size=3,stride=2)
    self.BatchNorm6 = torch.nn.BatchNorm2d(num_features=256)
    self.Conv7 = torch.nn.Conv2d(256, 512, kernel_size=3)
    self.BatchNorm7 = torch.nn.BatchNorm2d(num_features=512)
    self.Conv8 = torch.nn.Conv2d(512, 512, kernel_size=3, stride=2)
    self.BatchNorm8 = torch.nn.BatchNorm2d(num_features=512)

    self.hidden = torch.nn.Linear(in_features=131072, out_features=1024)
    self.final_classifier = torch.nn.Linear(in_features=1024, out_features=1)
    self.leakyReLU = torch.nn.LeakyReLU(negative_slope=0.2)

  def forward(self, x ):
    x = self.leakyReLU(self.Conv1(x))
    x = self.leakyReLU(self.BatchNorm2(self.Conv2(x)))
    x = self.leakyReLU(self.BatchNorm3(self.Conv3(x)))
    x = self.leakyReLU(self.BatchNorm4(self.Conv4(x)))
    x = self.leakyReLU(self.BatchNorm5(self.Conv5(x)))
    x = self.leakyReLU(self.BatchNorm6(self.Conv6(x)))
    x = self.leakyReLU(self.BatchNorm7(self.Conv7(x)))
    x = self.leakyReLU(self.BatchNorm8(self.Conv8(x)))
    x = x.reshape(-1)
    x = self.leakyReLU(self.hidden(x))
    return torch.sigmoid(x)

In [ ]:
class vgg_loss ():

  def __init__(self):
    super(vgg_loss, self).__init__()
    vgg = torchvision.models.vgg19()
    vgg = vgg.features

    # Freeze all vgg layers 
    for param in vgg.parameters():
      param.requires_grad = False 
    
    self.vgg = vgg[:27] 
    self.MSE = torch.nn.MSELoss()

  def __call__(self, input, target ):
    vgg_input = self.vgg(input)
    vgg_target = self.vgg(target)
    return self.MSE(vgg_input, vgg_target)


In [ ]:
disc = discrimator()
gen = generator()
disc_params = sum(p.numel() for p in disc.parameters())
gen_params = sum(p.numel() for p in gen.parameters())
print(disc_params)
print(gen_params)

140239425
484292


In [ ]:
loss = vgg_loss()
print(loss.vgg)

In [6]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict 

In [15]:
data_dir = '/content/drive/MyDrive/CSC413'

def get_images_x64(batch):
  filename = data_dir + "/x64/" + batch 
  d = unpickle(filename)
  x = d['data']
  x = np.dstack((x[:, :4096], x[:, 4096:8192], x[:, 8192:]))
  x = x.reshape((x.shape[0], 64, 64, 3))
  return x


def get_images_x16(filename):
  filename = data_dir + "/x16/" + batch 
  d = unpickle(filename)
  x = d['data']
  x = np.dstack((x[:, :256], x[:, 256:512], x[:, 512:]))
  x = x.reshape((x.shape[0], 16, 16, 3))
  return x  

In [34]:
%ls /content/drive/MyDrive/CSC413/x64

batch = 'train_data_batch_2'
X = get_images_x16( batch )
Y = get_images_x64( batch )

Imagenet64_train_part1.zip  train_data_batch_2  train_data_batch_7
Imagenet64_train_part2.zip  train_data_batch_3  train_data_batch_8
Imagenet64_val.zip          train_data_batch_4  train_data_batch_9
train_data_batch_1          train_data_batch_5  val_data
train_data_batch_10         train_data_batch_6


In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(1, 2))
plt.imshow(X[4, :])
plt.imshow(Y[4, :])

In [38]:
def gan_checkpoint(iteration, G, D, opts):
    """Saves the parameters of the generator G and discriminator D.
    """
    G_path = os.path.join(opts.checkpoint_dir, 'G.pkl')
    D_path = os.path.join(opts.checkpoint_dir, 'D.pkl')
    torch.save(G.state_dict(), G_path)
    torch.save(D.state_dict(), D_path)

def load_checkpoint(opts):
    """Loads the generator and discriminator models from checkpoints.
    """
    G_path = os.path.join(opts.load, 'G.pkl')
    D_path = os.path.join(opts.load, 'D_.pkl')

    G = DCGenerator(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
    D = DCDiscriminator(conv_dim=opts.d_conv_dim)

    G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
    D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

    if torch.cuda.is_available():
        G.cuda()
        D.cuda()
        print('Models moved to GPU.')

    return G, D




In [ ]:
"""

CREDITS: CSC413 PA4 DCGAN

"""
def gan_training_loop(dataloader, test_dataloader, opts):
    """Runs the training loop.
        * Saves checkpoint every opts.checkpoint_every iterations
        * Saves generated samples every opts.sample_every iterations
    """

    # Create generators and discriminators
    G = generator()
    D = discriminator()

    g_params = G.parameters()  # Get generator parameters
    d_params = D.parameters()  # Get discriminator parameters

    # Create optimizers for the generators and discriminators
    g_optimizer = optim.Adam(g_params, opts.lr, [opts.beta1, opts.beta2])
    d_optimizer = optim.Adam(d_params, opts.lr * 2., [opts.beta1, opts.beta2])

    train_iter = iter(dataloader)
    test_iter = iter(test_dataloader)

    # Get some fixed data from domains X and Y for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    fixed_noise = sample_noise(100, opts.noise_size)  # # 100 x noise_size x 1 x 1

    iter_per_epoch = len(train_iter)
    total_train_iters = opts.train_iters

 
    # adversarial_loss = torch.nn.BCEWithLogitsLoss()
    gp_weight = 1

    losses ={}
    losses['iteration'] = []
    losses['D_real_loss'] = []
    losses['D_fake_loss'] = []
    losses['G_loss'] = []


    try:
        for iteration in range(1, opts.train_iters + 1):

            # Reset data_iter for each epoch
            if iteration % iter_per_epoch == 0:
                train_iter = iter(dataloader)

            real_images, real_labels = train_iter.next()
            real_images, real_labels = to_var(real_images), to_var(real_labels).long().squeeze()

            for d_i in range(opts.d_train_iters):
                d_optimizer.zero_grad()


                # TODO modify to the new training loss
                m_rec = 1 /  (2 * real_images.shape[0])
                D_1 = (D(real_images) -1 )
                D_1 = D_1 ** 2 
                D_real_loss =  m_rec  * torch.sum(D_1) 
                noise = sample_noise(real_images.shape[0], opts.noise_size)
                fake_images = G(noise)
                D_fake_loss = D(fake_images) ** 2 
                D_fake_loss = m_rec * torch.sum(D_fake_loss)
          

                # ---- Gradient Penalty ----
                if opts.gradient_penalty:
                    alpha = torch.rand(real_images.shape[0], 1, 1, 1)
                    alpha = alpha.expand_as(real_images).cuda()
                    interp_images = Variable(alpha * real_images.data + (1 - alpha) * fake_images.data, requires_grad=True).cuda()
                    D_interp_output = D(interp_images)

                    gradients = torch.autograd.grad(outputs=D_interp_output, inputs=interp_images,
                                                    grad_outputs=torch.ones(D_interp_output.size()).cuda(),
                                                    create_graph=True, retain_graph=True)[0]
                    gradients = gradients.view(real_images.shape[0], -1)
                    gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

                    gp = gp_weight * gradients_norm.mean()
                else:
                    gp = 0.0

                D_total_loss = D_fake_loss + D_real_loss + gp 
                D_total_loss.backward()
                d_optimizer.step()


            g_optimizer.zero_grad()
            noise = sample_noise(real_images.shape[0], opts.noise_size)
            fake_images = G(noise)
            G_loss =  torch.sum( (D(fake_images) -1 ) ** 2) 
            G_loss = (1 / real_images.shape[0]) * G_loss 

            G_loss.backward()
            g_optimizer.step()
            
            if iteration % opts.log_step == 0:
                losses['iteration'].append(iteration)
                losses['D_real_loss'].append(D_real_loss.item())
                losses['D_fake_loss'].append(D_fake_loss.item())
                losses['G_loss'].append(G_loss.item())
                print('Iteration [{:4d}/{:4d}] | D_real_loss: {:6.4f} | D_fake_loss: {:6.4f} | G_loss: {:6.4f}'.format(
                    iteration, total_train_iters, D_real_loss.item(), D_fake_loss.item(), G_loss.item()))

            # Save the generated samples
            # if iteration % opts.sample_every == 0:
            #     gan_save_samples(G, fixed_noise, iteration, opts)

            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                gan_checkpoint(iteration, G, D, opts)

    except KeyboardInterrupt:
        print('Exiting early from training.')
        return G, D


    return G, D